In [2]:

import pandas as pd
import numpy as np

# ══════════════════════════════════════════════════════════════
# 1. LOAD DATA
# ══════════════════════════════════════════════════════════════

ca = pd.read_csv("data/credit_applications.csv")
tx = pd.read_csv("data/transactions_fintech.csv")
ua = pd.read_csv("data/user_activity_retention.csv")
uc = pd.read_csv("data/users_cohort.csv")

# Parse dates
ca["application_date"] = pd.to_datetime(ca["application_date"])
ca["approval_date"] = pd.to_datetime(ca["approval_date"], errors="coerce")
tx["timestamp"] = pd.to_datetime(tx["timestamp"])
ua["activity_date"] = pd.to_datetime(ua["activity_date"])
uc["signup_date"] = pd.to_datetime(uc["signup_date"])

# Filter to complete years 2024–2025
ca_full = ca[ca["application_date"].dt.year.isin([2024, 2025])].copy()
tx_full = tx[tx["timestamp"].dt.year.isin([2024, 2025])].copy()

print("=" * 60)
print("DATA LOADED")
print("=" * 60)
print(f"Credit applications (2024-2025): {len(ca_full):,}")
print(f"Credit applications (all):       {len(ca):,}")
print(f"Transactions (2024-2025):        {len(tx_full):,}")
print(f"User activity records:           {len(ua):,}")
print(f"User cohort records:             {len(uc):,}")


# ══════════════════════════════════════════════════════════════
# 2. EXECUTIVE SUMMARY METRICS
# ══════════════════════════════════════════════════════════════

print("\n" + "=" * 60)
print("EXECUTIVE SUMMARY")
print("=" * 60)

print(f"Total applications:       {len(ca_full):,}")
print(f"Unique users (credit):    {ca_full['user_id'].nunique():,}")
print(f"Total requested amount:   ${ca_full['requested_amount'].sum():,.0f}")
print(f"Avg requested amount:     ${ca_full['requested_amount'].mean():,.0f}")
print(f"Avg annual income:        ${ca_full['annual_income'].mean():,.0f}")
print(f"Overall approval rate:    {ca_full['is_approved'].mean() * 100:.1f}%")

approval_days = (ca_full["approval_date"] - ca_full["application_date"]).dt.days
print(f"Avg approval time:        {approval_days.mean():.1f} days")

approved = ca_full[ca_full["is_approved"] == True]
print(f"Default rate (approved):  {approved['default_flag'].mean() * 100:.2f}%")
print(f"Total defaults:           {approved['default_flag'].sum():,.0f}")

print(f"\nTotal transactions:       {len(tx_full):,}")
print(f"Total txn amount:         ${tx_full['amount'].sum():,.0f}")
print(f"Fraud count:              {tx_full['is_fraud'].sum():,}")
print(f"Fraud rate:               {tx_full['is_fraud'].mean() * 100:.2f}%")
print(f"Completed txns:           {(tx_full['status'] == 'completed').sum():,}")
print(f"Failed txns:              {(tx_full['status'] == 'failed').sum():,}")

refund_txns = tx_full[tx_full["transaction_type"] == "refund"]
print(f"\nRefund-type txns:         {len(refund_txns):,}")
print(f"Refund-type amount:       ${refund_txns['amount'].sum():,.0f}")
print(f"Refund % of total count:  {len(refund_txns) / len(tx_full) * 100:.1f}%")
print(f"  NOTE: These are transaction_type='refund', NOT status='refunded'")
print(f"  Actual refunded status count: {(tx_full['status'] == 'refunded').sum()}")

print(f"\nTotal users in cohort:    {len(uc):,}")
print(f"Avg LTV:                  ${uc['total_lifetime_value'].mean():,.2f}")


# ══════════════════════════════════════════════════════════════
# 3. YEAR-OVER-YEAR COMPARISON
# ══════════════════════════════════════════════════════════════

print("\n" + "=" * 60)
print("YEAR-OVER-YEAR COMPARISON (2024 vs 2025)")
print("=" * 60)

ca_full["year"] = ca_full["application_date"].dt.year
yoy = ca_full.groupby("year").agg(
    apps=("application_id", "count"),
    total_req=("requested_amount", "sum"),
    avg_req=("requested_amount", "mean"),
    approval_rate=("is_approved", lambda x: x.mean() * 100),
    avg_income=("annual_income", "mean"),
).round(2)

print(yoy.to_string())
print(f"\nYoY apps change:      {(yoy.loc[2025, 'apps'] / yoy.loc[2024, 'apps'] - 1) * 100:.2f}%")
print(f"YoY total_req change: {(yoy.loc[2025, 'total_req'] / yoy.loc[2024, 'total_req'] - 1) * 100:.2f}%")
print(f"YoY approval change:  {yoy.loc[2025, 'approval_rate'] - yoy.loc[2024, 'approval_rate']:.2f}pp")


# ══════════════════════════════════════════════════════════════
# 4. QUARTERLY TRENDS
# ══════════════════════════════════════════════════════════════

print("\n" + "=" * 60)
print("QUARTERLY APPLICATION TRENDS")
print("=" * 60)

ca_full["quarter"] = ca_full["application_date"].dt.to_period("Q")
q = ca_full.groupby("quarter").agg(
    apps=("application_id", "count"),
    total_req=("requested_amount", "sum"),
    avg_req=("requested_amount", "mean"),
    approval_rate=("is_approved", lambda x: x.mean() * 100),
    avg_income=("annual_income", "mean"),
).round(2)
q["qoq_apps_pct"] = (q["apps"].pct_change() * 100).round(2)

print(q.to_string())


# ══════════════════════════════════════════════════════════════
# 5. MONTHLY TRENDS
# ══════════════════════════════════════════════════════════════

print("\n" + "=" * 60)
print("MONTHLY APPLICATION TRENDS")
print("=" * 60)

ca_full["month"] = ca_full["application_date"].dt.to_period("M")
m = ca_full.groupby("month").agg(
    apps=("application_id", "count"),
    total_req=("requested_amount", "sum"),
    avg_req=("requested_amount", "mean"),
    approval_rate=("is_approved", lambda x: x.mean() * 100),
).round(2)

print(m.to_string())


# ══════════════════════════════════════════════════════════════
# 6. CREDIT SCORE BAND ANALYSIS
# ══════════════════════════════════════════════════════════════

print("\n" + "=" * 60)
print("CREDIT SCORE BAND ANALYSIS")
print("=" * 60)

bins = [0, 600, 700, 763, 800, 850, 901]
labels = ["<600", "600-699", "700-762", "763-799", "800-849", "850+"]
ca_full["score_band"] = pd.cut(ca_full["credit_score"], bins=bins, labels=labels, right=False)

sb = ca_full.groupby("score_band", observed=True).agg(
    apps=("application_id", "count"),
    approval_rate=("is_approved", lambda x: x.mean() * 100),
    avg_req=("requested_amount", "mean"),
    total_req=("requested_amount", "sum"),
).round(2)
sb["pct_of_total"] = (sb["apps"] / sb["apps"].sum() * 100).round(1)

# Default rate by score band
approved_bands = ca_full[ca_full["is_approved"] == True].copy()
approved_bands["score_band"] = pd.cut(
    approved_bands["credit_score"], bins=bins, labels=labels, right=False
)
dr = approved_bands.groupby("score_band", observed=True).agg(
    defaults=("default_flag", "sum"),
    approved_count=("application_id", "count"),
)
dr["default_rate"] = (dr["defaults"] / dr["approved_count"] * 100).round(2)
sb = sb.join(dr[["default_rate"]])

print(sb.to_string())
print("\n>>> NOTE: Default rate is ~5% across ALL score bands — model may lack")
print("    discriminatory power for predicting defaults.")


# ══════════════════════════════════════════════════════════════
# 7. LOAN PURPOSE ANALYSIS
# ══════════════════════════════════════════════════════════════

print("\n" + "=" * 60)
print("LOAN PURPOSE ANALYSIS")
print("=" * 60)

lp = ca_full.groupby("loan_purpose").agg(
    apps=("application_id", "count"),
    approval_rate=("is_approved", lambda x: x.mean() * 100),
    avg_req=("requested_amount", "mean"),
    total_req=("requested_amount", "sum"),
).round(2)
lp["pct_of_total"] = (lp["apps"] / lp["apps"].sum() * 100).round(1)

# Default rate by loan purpose
dr_lp = approved.groupby("loan_purpose").agg(
    defaults=("default_flag", "sum"),
    approved_count=("application_id", "count"),
)
dr_lp["default_rate"] = (dr_lp["defaults"] / dr_lp["approved_count"] * 100).round(2)
lp = lp.join(dr_lp[["default_rate"]])

print(lp.to_string())


# ══════════════════════════════════════════════════════════════
# 8. AGE GROUP ANALYSIS (merged with cohort)
# ══════════════════════════════════════════════════════════════

print("\n" + "=" * 60)
print("AGE GROUP ANALYSIS")
print("=" * 60)

merged = ca_full.merge(
    uc[["user_id", "age_group", "account_type", "signup_channel", "country"]],
    on="user_id",
    how="left",
)

ag = merged.groupby("age_group").agg(
    apps=("application_id", "count"),
    approval_rate=("is_approved", lambda x: x.mean() * 100),
    avg_income=("annual_income", "mean"),
    total_income=("annual_income", "sum"),
    avg_req=("requested_amount", "mean"),
    total_req=("requested_amount", "sum"),
).round(2)
ag["pct_of_total"] = (ag["apps"] / ag["apps"].sum() * 100).round(1)

print(ag.to_string())

# LTV by age group
print("\nLTV by age group:")
ag_ltv = uc.groupby("age_group")["total_lifetime_value"].agg(["mean", "sum", "count"]).round(2)
ag_ltv.columns = ["avg_ltv", "total_ltv", "users"]
print(ag_ltv.to_string())


# ══════════════════════════════════════════════════════════════
# 9. ACCOUNT TYPE ANALYSIS
# ══════════════════════════════════════════════════════════════

print("\n" + "=" * 60)
print("ACCOUNT TYPE ANALYSIS")
print("=" * 60)

at = merged.groupby("account_type").agg(
    apps=("application_id", "count"),
    approval_rate=("is_approved", lambda x: x.mean() * 100),
    avg_req=("requested_amount", "mean"),
    total_req=("requested_amount", "sum"),
).round(2)
at["pct_of_total"] = (at["apps"] / at["apps"].sum() * 100).round(1)

print("Applications by account type:")
print(at.to_string())

print("\nLTV by account type:")
ltv_at = uc.groupby("account_type")["total_lifetime_value"].agg(["mean", "sum", "count"]).round(2)
ltv_at.columns = ["avg_ltv", "total_ltv", "users"]
print(ltv_at.to_string())
print("\n>>> NOTE: LTV is virtually identical across all account types (~$2,500).")
print("    Premium/enterprise tiers do not produce higher lifetime value.")


# ══════════════════════════════════════════════════════════════
# 10. EMPLOYMENT STATUS
# ══════════════════════════════════════════════════════════════

print("\n" + "=" * 60)
print("EMPLOYMENT STATUS ANALYSIS")
print("=" * 60)

es = ca_full.groupby("employment_status").agg(
    apps=("application_id", "count"),
    approval_rate=("is_approved", lambda x: x.mean() * 100),
    avg_income=("annual_income", "mean"),
    avg_req=("requested_amount", "mean"),
).round(2)
es["pct_of_total"] = (es["apps"] / es["apps"].sum() * 100).round(1)

print(es.to_string())


# ══════════════════════════════════════════════════════════════
# 11. TRANSACTION ANALYSIS
# ══════════════════════════════════════════════════════════════

print("\n" + "=" * 60)
print("TRANSACTION TYPE DISTRIBUTION")
print("=" * 60)

tt = tx_full.groupby("transaction_type").agg(
    count=("transaction_id", "count"),
    amount=("amount", "sum"),
).round(2)
tt["pct_count"] = (tt["count"] / tt["count"].sum() * 100).round(1)
tt["pct_amount"] = (tt["amount"] / tt["amount"].sum() * 100).round(1)

print(tt.to_string())

print("\nTransaction status distribution:")
print(tx_full["status"].value_counts().to_string())

print(f"\n>>> IMPORTANT: 'refund' is a transaction_type, not a status.")
print(f"    There are ZERO records with status='refunded'.")
print(f"    The dashboard's '20% refund rate' = proportion of refund-type transactions.")


# ══════════════════════════════════════════════════════════════
# 12. TRANSACTIONS BY COUNTRY
# ══════════════════════════════════════════════════════════════

print("\n" + "=" * 60)
print("TRANSACTIONS BY COUNTRY")
print("=" * 60)

tc = tx_full.groupby("country").agg(
    count=("transaction_id", "count"),
    amount=("amount", "sum"),
    fraud=("is_fraud", "sum"),
).round(2)
tc["pct_count"] = (tc["count"] / tc["count"].sum() * 100).round(1)
tc["fraud_rate"] = (tc["fraud"] / tc["count"] * 100).round(2)
tc["avg_txn"] = (tc["amount"] / tc["count"]).round(2)

print(tc.sort_values("amount", ascending=False).to_string())


# ══════════════════════════════════════════════════════════════
# 13. TRANSACTIONS BY MERCHANT CATEGORY
# ══════════════════════════════════════════════════════════════

print("\n" + "=" * 60)
print("TRANSACTIONS BY MERCHANT CATEGORY")
print("=" * 60)

mc = tx_full.groupby("merchant_category").agg(
    count=("transaction_id", "count"),
    amount=("amount", "sum"),
    fraud=("is_fraud", "sum"),
).round(2)
mc["pct_amount"] = (mc["amount"] / mc["amount"].sum() * 100).round(1)
mc["fraud_rate"] = (mc["fraud"] / mc["count"] * 100).round(2)

print(mc.to_string())


# ══════════════════════════════════════════════════════════════
# 14. TRANSACTIONS BY PAYMENT METHOD
# ══════════════════════════════════════════════════════════════

print("\n" + "=" * 60)
print("TRANSACTIONS BY PAYMENT METHOD")
print("=" * 60)

pm = tx_full.groupby("payment_method").agg(
    count=("transaction_id", "count"),
    amount=("amount", "sum"),
    fraud=("is_fraud", "sum"),
).round(2)
pm["fraud_rate"] = (pm["fraud"] / pm["count"] * 100).round(2)

print(pm.to_string())


# ══════════════════════════════════════════════════════════════
# 15. QUARTERLY TRANSACTION TRENDS
# ══════════════════════════════════════════════════════════════

print("\n" + "=" * 60)
print("QUARTERLY TRANSACTION TRENDS")
print("=" * 60)

tx_full["quarter"] = tx_full["timestamp"].dt.to_period("Q")
tq = tx_full.groupby("quarter").agg(
    count=("transaction_id", "count"),
    amount=("amount", "sum"),
    fraud=("is_fraud", "sum"),
    completed=("status", lambda x: (x == "completed").sum()),
    failed=("status", lambda x: (x == "failed").sum()),
).round(2)
tq["fraud_rate"] = (tq["fraud"] / tq["count"] * 100).round(2)

print(tq.to_string())


# ══════════════════════════════════════════════════════════════
# 16. COMPLETED FRAUD INVESTIGATION
# ══════════════════════════════════════════════════════════════

print("\n" + "=" * 60)
print("COMPLETED FRAUD TRANSACTIONS")
print("=" * 60)

completed_fraud = tx_full[(tx_full["status"] == "completed") & (tx_full["is_fraud"] == True)]
print(f"Count:  {len(completed_fraud):,}")
print(f"Amount: ${completed_fraud['amount'].sum():,.0f}")
print(f"\nThese transactions passed through as completed despite being flagged as fraud.")


# ══════════════════════════════════════════════════════════════
# 17. USER ACTIVITY — DEVICE ANALYSIS
# ══════════════════════════════════════════════════════════════

print("\n" + "=" * 60)
print("USER ACTIVITY BY DEVICE")
print("=" * 60)

dev = ua.groupby("device_type").agg(
    sessions=("activity_id", "count"),
    total_duration=("session_duration_minutes", "sum"),
    avg_duration=("session_duration_minutes", "mean"),
    avg_pages=("pages_viewed", "mean"),
    avg_actions=("actions_taken", "mean"),
).round(2)
dev["pct_sessions"] = (dev["sessions"] / dev["sessions"].sum() * 100).round(1)

print(dev.to_string())


# ══════════════════════════════════════════════════════════════
# 18. USER ACTIVITY — DAY OF WEEK
# ══════════════════════════════════════════════════════════════

print("\n" + "=" * 60)
print("USER ACTIVITY BY DAY OF WEEK")
print("=" * 60)

ua["dow"] = ua["activity_date"].dt.day_name()
order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
dow = (
    ua.groupby("dow")
    .agg(
        sessions=("activity_id", "count"),
        avg_duration=("session_duration_minutes", "mean"),
        total_duration=("session_duration_minutes", "sum"),
        avg_pages=("pages_viewed", "mean"),
        avg_actions=("actions_taken", "mean"),
    )
    .round(2)
    .reindex(order)
)

print(dow.to_string())
print("\n>>> NOTE: Sessions are evenly distributed across all days (~42.5K–43.3K).")
print("    There is no meaningful Sunday/Monday peak.")


# ══════════════════════════════════════════════════════════════
# 19. CORRELATION ANALYSIS
# ══════════════════════════════════════════════════════════════

print("\n" + "=" * 60)
print("CORRELATION ANALYSIS")
print("=" * 60)

corr_pages_actions = ua["pages_viewed"].corr(ua["actions_taken"])
corr_duration_actions = ua["session_duration_minutes"].corr(ua["actions_taken"])
corr_duration_pages = ua["session_duration_minutes"].corr(ua["pages_viewed"])

print(f"pages_viewed vs actions_taken:        r = {corr_pages_actions:.4f}")
print(f"session_duration vs actions_taken:     r = {corr_duration_actions:.4f}")
print(f"session_duration vs pages_viewed:      r = {corr_duration_pages:.4f}")
print("\n>>> NOTE: All correlations are essentially zero.")
print("    Pages viewed does NOT predict actions taken.")
print("    Session duration does NOT predict engagement.")


# ══════════════════════════════════════════════════════════════
# 20. LTV ANALYSIS BY SEGMENT
# ══════════════════════════════════════════════════════════════

print("\n" + "=" * 60)
print("LIFETIME VALUE BY SEGMENT")
print("=" * 60)

print("\nBy signup channel:")
sc = uc.groupby("signup_channel").agg(
    users=("user_id", "count"),
    avg_ltv=("total_lifetime_value", "mean"),
    total_ltv=("total_lifetime_value", "sum"),
).round(2)
sc["pct_users"] = (sc["users"] / sc["users"].sum() * 100).round(1)
print(sc.to_string())

print("\nBy country:")
cc = uc.groupby("country").agg(
    users=("user_id", "count"),
    avg_ltv=("total_lifetime_value", "mean"),
    total_ltv=("total_lifetime_value", "sum"),
).round(2)
cc["pct_users"] = (cc["users"] / cc["users"].sum() * 100).round(1)
print(cc.to_string())

print("\nBy verification status:")
ver = uc.groupby("is_verified").agg(
    users=("user_id", "count"),
    avg_ltv=("total_lifetime_value", "mean"),
).round(2)
print(ver.to_string())
print("\n>>> NOTE: LTV is ~$2,501 across ALL segments. No meaningful differentiation.")


# ══════════════════════════════════════════════════════════════
# 21. CONVERSION FUNNEL (from dashboard event data)
# ══════════════════════════════════════════════════════════════

print("\n" + "=" * 60)
print("CONVERSION FUNNEL (from dashboard)")
print("=" * 60)

funnel = [
    ("app_open", 128000),
    ("view_homepage", 108000),
    ("view_product", 90000),
    ("add_to_cart", 51000),
    ("begin_checkout", 38000),
    ("add_payment_method", 32000),
    ("submit_order", 28000),
    ("order_confirmed", 25000),
]

print(f"{'Stage':<25} {'Count':>8} {'Step Conv.':>12} {'Overall':>10}")
print("-" * 58)
for i, (stage, count) in enumerate(funnel):
    prev_pct = f"{count / funnel[i - 1][1] * 100:.1f}%" if i > 0 else "—"
    top_pct = f"{count / funnel[0][1] * 100:.1f}%"
    marker = " <<< BIGGEST DROP" if i == 3 else ""
    print(f"{stage:<25} {count:>8,} {prev_pct:>12} {top_pct:>10}{marker}")

print("\n>>> NOTE: The biggest drop-off is view_product -> add_to_cart (43.3% loss).")
print("    This is the primary UX optimisation target, not checkout.")


FileNotFoundError: [Errno 2] No such file or directory: 'data/credit_applications.csv'